In [1]:
import pandas as pd
df = pd.read_csv('responses.csv')
df.head()

,Category,Prompt,Model_Response_Summary,Failure_Type
0,Ambiguous,Show me something different.,Suggested 'Everything Everywhere All at Once' ...,Successful Reinterpretation
1,Ambiguous,I'm in the mood for a classic vibe.,Suggested 'Casablanca'.,Successful Reinterpretation
2,Ambiguous,I want a movie that feels like a Tuesday.,"Suggested 'Paterson' (mundane, poetic).",Successful Reinterpretation
3,Ambiguous,Something good.,Suggested 'Shawshank Redemption' (crowd pleaser).,Confident Failure
4,Ambiguous,Give me a movie for when I'm thinking.,Suggested 'Arrival' (linguistics/time).,Successful Reinterpretation


In [2]:
df.groupby('Category')['Failure_Type'].value_counts()

Category       Failure_Type               
Ambiguous      Successful Reinterpretation    12
               Confident Failure               3
Contradictory  Successful Reinterpretation    13
               Hallucination                   2
Nonsensical    Successful Reinterpretation    13
               Constraint Ignored              2
Name: count, dtype: int64